 # SQL Injection Demo

# Setting up connection to db

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('demonstration.db')
cursor = conn.cursor()


In [2]:
def print_users(result):
    if (len(result) == 0):
        print("No data retrieved")
    else:
        print(pd.DataFrame(result, columns=["username", "first name", "last name", "password"]))
    

# Example from presentation (Unsafe)

In [3]:
def unsafe_find_user_by_fName(fName):
    # Both of these query creations are unsafe
    query = "SELECT username, fName, lName, password FROM users where fName = '" + fName + "'"
    #query = "SELECT username, fName, lName, password FROM users where fName = '%s'" % fName
    print("Resulting query: " + query + "\n")
    try:
        result = cursor.execute(query).fetchall()
        print_users(result)
    except sqlite3.OperationalError as e:
        print("Error thrown -  " + str(e))

## Intended usage

In [4]:
unsafe_find_user_by_fName("David")

Resulting query: SELECT username, fName, lName, password FROM users where fName = 'David'

            username first name last name        password
0  ComputerWhisperer      David    Stotts  i032r23b2o1kwe


## Intended, non malicious usage

In [5]:
unsafe_find_user_by_fName("Dav'id")

Resulting query: SELECT username, fName, lName, password FROM users where fName = 'Dav'id'

Error thrown -  near "id": syntax error


## Malicious usage

In [6]:
unsafe_find_user_by_fName("David' OR 1=1--")

Resulting query: SELECT username, fName, lName, password FROM users where fName = 'David' OR 1=1--'

            username first name last name        password
0         CompSciKid      James     Jones      password01
1        GuyWhoCodes     Steven     Smith          123456
2  ComputerWhisperer      David    Stotts  i032r23b2o1kwe
3     TheCodeBringer     Dav'id  Richards        drowssap





# Preventing SQL Injection

In [8]:
def safe_find_user_by_fName(fName):
    # Technically a parameterized query, not a prepared statement, but for the purpose of demonstration, works the same
    # Essentially creates a prepared statement and immediately fills in the fields, as opposed to doing it as separate
    # Operations
    query = "SELECT username, fName, lName, password FROM users where fName =?"
    result = cursor.execute(query, (fName,)).fetchall()
    print_users(result)

# Intended usage

In [9]:
safe_find_user_by_fName("David")

            username first name last name        password
0  ComputerWhisperer      David    Stotts  i032r23b2o1kwe


# Intended, non malicious usage

In [10]:
safe_find_user_by_fName("Dav'id")

         username first name last name  password
0  TheCodeBringer     Dav'id  Richards  drowssap


# Malicious usage

In [11]:
safe_find_user_by_fName("David OR 1=1--")

No data retrieved


In [12]:
conn.close()